In [1]:
pip install pyspark


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

# Start SparkSession
spark = SparkSession.builder.appName("CSV Load Example").master("local[*]").getOrCreate()

# Load CSV file into DataFrame
df = spark.read.csv("C:/Users/andre/OneDrive/Dokument/vscode/Examens/aapl_10min_2yr.csv", header=True, inferSchema=True)
# Show the first few rows
df.show()

# Print the schema (column types)
df.printSchema()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.